@author:         leo.unite        Diego.Casas       Romain.Salmeron

# Chargement des librairies utilisées
`pip install pandas selenium bs4`

In [2]:
import pandas
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# Importation des données

In [3]:
df = pandas.read_csv("export_for_search_results.csv")
df

,adresse,latitude,longitude,result_score,result_score_next,result_label,result_type,result_id,result_housenumber,result_name,result_street,result_postcode,result_city,result_context,result_citycode,result_oldcitycode,result_oldcity,result_district,result_status
0,25 RUE LOUVRE,48.797567,2.180235,0.786110,0.782672,25 Rue du Louvre 78220 Viroflay,housenumber,78686_0800_00025,25,25 Rue du Louvre,Rue du Louvre,78220,Viroflay,"78, Yvelines, Île-de-France",78686,NaN,NaN,NaN,ok
1,1 RUE PIERRE LESCOT,42.693743,2.916540,0.976288,0.969725,1 Rue Pierre Lescot 66000 Perpignan,housenumber,66136_3977_00001,1,1 Rue Pierre Lescot,Rue Pierre Lescot,66000,Perpignan,"66, Pyrénées-Orientales, Occitanie",66136,NaN,NaN,NaN,ok
2,196 RUE SAINT HONORE,49.885385,2.285127,0.982789,0.973353,196 Rue Saint-Honoré 80000 Amiens,housenumber,80021_7230_00196,196,196 Rue Saint-Honoré,Rue Saint-Honoré,80000,Amiens,"80, Somme, Hauts-de-France",80021,NaN,NaN,NaN,ok
3,7 RUE ARGENTEUIL,48.942422,2.371445,0.867374,0.865396,7 Rue d'Argenteuil 93200 Saint-Denis,housenumber,93066_0055_00007,7,7 Rue d'Argenteuil,Rue d'Argenteuil,93200,Saint-Denis,"93, Seine-Saint-Denis, Île-de-France",93066,NaN,NaN,NaN,ok
4,20 RUE PYRAMIDES,50.627747,3.059359,0.786554,0.780982,20 Rue des Pyramides 59000 Lille,housenumber,59350_7274_00020,20,20 Rue des Pyramides,Rue des Pyramides,59000,Lille,"59, Nord, Hauts-de-France",59350,59350.0,Lille,NaN,ok
5,22 AV OPERA,48.866134,2.334474,0.463536,0.325875,22 Avenue de l'Opéra 75001 Paris,housenumber,75101_6904_00022,22,22 Avenue de l'Opéra,Avenue de l'Opéra,75001,Paris,"75, Paris, Île-de-France",75101,NaN,NaN,Paris 1er Arrondissement,ok
6,15 RUE MOLIERE,43.592486,1.426017,0.979675,0.979605,15 Rue Molière 31300 Toulouse,housenumber,31555_5996_00015,15,15 Rue Molière,Rue Molière,31300,Toulouse,"31, Haute-Garonne, Occitanie",31555,NaN,NaN,NaN,ok
7,51 RUE PETITS CHAMPS,48.685312,2.347556,0.811155,0.807305,51 Rue des Petits Champs 91600 Savigny-sur-Orge,housenumber,91589_4380_00051,51,51 Rue des Petits Champs,Rue des Petits Champs,91600,Savigny-sur-Orge,"91, Essonne, Île-de-France",91589,NaN,NaN,NaN,ok
8,16 RUE SAINT ROCH,45.650070,0.161635,0.977025,0.976579,16 Rue Saint Roch 16000 Angoulême,housenumber,16015_3780_00016,16,16 Rue Saint Roch,Rue Saint Roch,16000,Angoulême,"16, Charente, Nouvelle-Aquitaine",16015,NaN,NaN,NaN,ok
9,14 RUE MARCHE SAINT HONORE,48.866425,2.331500,0.863812,0.624264,14 Rue du Marché Saint-Honoré 75001 Paris,housenumber,75101_6034_00014,14,14 Rue du Marché Saint-Honoré,Rue du Marché Saint-Honoré,75001,Paris,"75, Paris, Île-de-France",75101,NaN,NaN,Paris 1er Arrondissement,ok


# Première étape : configurer le navigateur
Pour que `selenium` fonctionne comme ce qui suit, sous Windows, il est nécessaire d’avoir télécharger l’exécutable [chromedriver](https://storage.googleapis.com/chrome-for-testing-public/132.0.6834.110/win64/chromedriver-win64.zip) dans le répertoire de travail de Python. Vous pouvez récupérer les versions stables sur [cette page](https://googlechromelabs.github.io/chrome-for-testing/#stable) (copier-coller le lien HTML dans la barre du navigateur pour télécharger le fichier).

`pip install selenium` in Terminal

In [5]:
driver = webdriver.Chrome()

## Création de l’URL à récupérer
On créé l’URL de la page qu’on souhaite récupérer. Pour faire une recherche dans Google Maps, il est préférable de mettre le nom de l’établissement, ainsi que les coordonnées géographiques de celui-ci.

In [6]:
base_url = "https://www.google.com/maps/search/"
place_info = "IUT+paris+rives+de+seine"
comp_url = "/@48.8489968,2.3125954,12z"

url = base_url + place_info + comp_url
url

'https://www.google.com/maps/search/IUT+paris+rives+de+seine/@48.8489968,2.3125954,12z'

# Premier lancement du navigateur
Pour rappel, **NE FERMEZ PAS LA FENETRE** mais cliquer sur Tout autoriser

In [7]:
driver.get(url)

## Fonction pour création de l'URL
On reprend la fonction vue dans la séance 1, avec une petite amélioration pour mettre le numéro de l'adresse en entier.

In [8]:
def f(v, fin = False):
    r = ""
    if not(pandas.isna(v)):
        if isinstance(v, float):
            r += str(int(v))
        else:
            r += str(v)
        if not(fin):
            r += "+"
    return r

### Code de recherche

In [10]:
# Code de recherche
site = []
for i in range(df.shape[0]):
    place_info = f(df["result_name"][i]) + f(df["result_street"][i]) + f(df["result_postcode"][i]) + f(df["result_city"][i], True)
    comp_url = "/@" +  str(df["latitude"][i]) + "," + str(df["longitude"][i]) + ",12z"
    url = base_url + place_info + comp_url
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html)
    results = soup.select("a[aria-label*='Site Web']")
    href = [None]
    if results != []:
        href += [results[0]["href"]]
    site += href
    time.sleep(1)

### Affichage du résultat et stockage dans les données

In [11]:
site
df["site"] = site
df.to_csv("BDCOM_2023(in)_avec_site.csv")